In [1]:
import grequests

In [2]:
import requests
import urllib.parse

In [3]:
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
import os, re
from time import sleep, time

### Get all vehicle details links

In [5]:
all_links = []

def get_links(soup):
    tags = soup.findAll('a',attrs={'class':'shop-srp-listings__listing'})
    return ['https://www.cars.com' + x['href'] for x in tags]
    
    
for i in range(0, 52):
    print(i)
    url = f'https://www.cars.com/for-sale/searchresults.action/?page={i}&perPage=100&rd=75&searchSource=GN_REFINEMENT&shippable-dealers-checkbox=true&showMore=false&sort=relevance&zc=35203&localVehicles=false'  
    r = requests.get(url)
    soup = BeautifulSoup(r.content)
    links = get_links(soup)
    
    for l in links:
        all_links.append(l)



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51


In [6]:
print(len(all_links) , len(set(all_links)))

5200 4945


In [7]:
with open('links_alabama.txt', 'w') as f:
    for x in list(set(all_links)):
        f.write('%s\n' %x)

### Class uses grequests to parallelize requests to cars.com

In [ ]:
# lst = ['https://www.cars.com/vehicledetail/detail/764209894/overview/',
#        'https://www.cars.com/vehicledetail/detail/765560013/overview/',
#        'https://www.cars.com/vehicledetail/detail/761675529/overview/',
#       'https://www.cars.com/vehicledetail/detail/759093860/overview/',
#       'https://www.cars.com/vehicledetail/detail/759217275/overview/',
#       'https://www.cars.com/vehicledetail/detail/759684992/overview/',
#       'https://www.cars.com/vehicledetail/detail/764048732/overview/',
#       'https://www.cars.com/vehicledetail/detail/761718085/overview/',
#       'https://www.cars.com/vehicledetail/detail/764655488/overview/',
#       'https://www.cars.com/vehicledetail/detail/760249404/overview/',
#       'https://www.cars.com/vehicledetail/detail/760249342/overview/',
#       ]

df = pd.read_csv('links_py.txt', header=None)
lst = df[0].tolist()
lst = lst[:50]

class Test():
    def __init__(self, sublst, size):
        self.urls = sublst
        self.size = size

    def exception(self, request, exception):
        print ("Problem: {}: {}".format(request.url, exception))

    def async_req(self):
        return grequests.map((grequests.get(u) for u in self.urls), 
                             exception_handler=self.exception, size= self.size)

    def collate_responses(self, results):
        return [x.text for x in results]



#### Saves the raw html after a sucessful request

In [8]:

step = 10
all_r, failed = [], []

def write_html(html, n, url):
    st = url.find('/detail/')
    end = url.find('/overview/')
    _id = url[st+len('/detail/'):end]
    
    with open(f'/home/jeferson/personal_projects/scraping_cars/results/ny/{n}-{_id}', 'wb') as f:
        f.write(html)
    
n = 0
for i in range(step-1, len(lst)+step , step):
    st = time()
    test_inst = Test(lst[i-step+1:i+1], step)
    
    res = test_inst.async_req()
    print(res)
    for _res in res:
        if _res.status_code != 200:
            failed.append(_res.url)
        else:
            write_html(_res.content, n, _res.url)
    n += 1
    print(time()-st,'\n')
    
#     print(lst[i-step+1:i+1])



NameError: name 'lst' is not defined